# DATASCI W261: Machine Learning at Scale
## Assignment Week 7
Miki Seltzer (miki.seltzer@berkeley.edu)<br>
W261-2, Spring 2016<br>
Submission: 

# HW 7.0: Shortest path graph distances (toy networks)

In this part of your assignment you will develop the base of your code for the week.

Write MRJob classes to find shortest path graph distances, as described in the lectures. In addition to finding the distances, your code should also output a distance-minimizing path between the source and target. Work locally for this part of the assignment, and use both of the undirected and directed toy networks.

![Toy networks](toy_graphs.png)

To proof you code's function, run the following jobs

- shortest path in the undirected network from node 1 to node 4
Solution: 1,5,4 

- shortest path in the directed network from node 1 to node 5
Solution: 1,2,4,5

and report your output -- make sure it is correct!

## Initialize graph structure

We have the graph encoded as an adjacency list, but we need to keep track of state in each iteration
- shortest distance from start node
- node state (unvisited, queued, visited)

In [1]:
# We will need these so we can reload modules as we modify them
%load_ext autoreload
%autoreload 2

In [2]:
import sys

def initiate_graph(filename, startNode):
    inGraph = open(filename, 'r')
    outGraph = open(filename.replace('.', '_temp.'), 'w')

    for node in inGraph:
        fields = node.strip().split('\t')
        nodeName = fields[0]
        nodeNeighbors = eval(fields[1])
        if nodeName == startNode:
            outGraph.write(nodeName + '\t' + str([nodeNeighbors, 0, 'Q']) + '\n')
        else:
            outGraph.write(nodeName + '\t' + str([nodeNeighbors, sys.maxint, 'U']) + '\n')

    inGraph.close()
    outGraph.close()

In [3]:
%%writefile MRJob_ShortestPath.py
from mrjob.job import MRJob
from mrjob.step import MRStep

class shortestPath(MRJob):
    
    # Specify some custom options so we only have to write one MRJob class for each join
    def configure_options(self):
        super(join, self).configure_options()
        self.add_passthrough_option('--startNode', default='1')
    
    def mapper(self, _, line):
        yield None, line
        
        
    """
    Multistep pipeline definition
    """
#     def steps(self):
#         return [
#                 MRStep()
#             ]
    
if __name__ == '__main__':
    shortestPath.run()

Overwriting MRJob_ShortestPath.py


In [30]:
from MRJob_ShortestPath import shortestPath

def findShortestPath(filename, startNode):

    initiate_graph(filename, startNode)
    temp_filename = filename.replace('.', '_temp.')
    
    mr_job = join(args=[temp_filename, '--no-strict-protocols'])
    output = []

    with mr_job.make_runner() as runner: 
        # Run MRJob
        runner.run()

        # Write stream_output to file
        for line in runner.stream_output():
            output.append(mr_job.parse_output_line(line))
    
    return output

findShortestPath('undirected_toy.txt', '1')

ImportError: No module named mrjob.job